In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import k, e, pi
from pathlib import Path
import string

from matplotlib import rcParams as rc
# import gridspec
import matplotlib.gridspec as gridspec

listAlphabet = list(string.ascii_lowercase)
rc["mathtext.fontset"] = "stix"
rc["font.family"] = "STIXGeneral"

# Enable LaTeX rendering
plt.rcParams['text.usetex'] = True
plt.rc("text", usetex=True)

plt.style.use("/home/petronio/Documents/varie/style_fp.mplstyle")

colors = ["orange", "darkgreen", "r","purple","b", "black", "cyan", "gray", "pink", "brown", "magenta", "teal", "gold", "lightblue", "lightgreen"]
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)


# IMAGES WIDTH
image_width_small = 3.54  # inches
image_width_large = 7.48  # inches
plt.rcParams["figure.figsize"] = (image_width_small, 3)


In [ ]:
mu = 3.986e14  # m^3/s^2
R_earth = 6371e3  # m
altitudes = np.linspace(100e3, 2000e3, 100)  # m
orbital_speeds = np.sqrt(mu / (R_earth + altitudes))

In [ ]:
plt.figure()
plt.plot(altitudes / 1e3, orbital_speeds / 1e3)
plt.xlabel("Altitude [km]")
plt.ylabel("Orbital Speed [km/s]")
plt.title("Orbital Speed vs Altitude")
plt.grid()
plt.xlim(0, 2000)
# plt.savefig('orbital_speed_vs_altitude.png')

In [ ]:
import pandas as pd

altitude = 250e3  # m

comp_data = pd.read_csv("comp_atm_ready.txt", sep="\t")
print(comp_data.columns)
comp_data = comp_data[comp_data["Heit(km)"] == 250]
O2_density = comp_data["O2den(m-3)"].values[0]  # m^-3
N2_density = comp_data["N2den(m-3)"].values[0]  # m^-3
O_density = comp_data["Oden(m-3)"].values[0]  # m^-3
N_density = comp_data["Nden(m-3)"].values[0]  # m^-3

AREA = 1.0  # m^2
orbital_speeds = np.sqrt(mu / (R_earth + altitude))

O2_injection_rate = orbital_speeds * O2_density * AREA  # particles/s
N2_injection_rate = orbital_speeds * N2_density * AREA  # particles/s
O_injection_rate = orbital_speeds * O_density * AREA  # particles/s
N_injection_rate = orbital_speeds * N_density * AREA  # particles/s

print(
    "O2 injection rate:",
    O2_injection_rate,
    comp_data["Q_O2(s-1)"].values[0],
    (O2_injection_rate - comp_data["Q_O2(s-1)"].values[0])
    / comp_data["Q_O2(s-1)"].values[0],
)
print(
    "N2 injection rate:",
    N2_injection_rate,
    comp_data["Q_N2(s-1)"].values[0],
    (N2_injection_rate - comp_data["Q_N2(s-1)"].values[0])
    / comp_data["Q_N2(s-1)"].values[0],
)
print(
    "O injection rate:",
    O_injection_rate,
    comp_data["Q_O(s-1)"].values[0],
    (O_injection_rate - comp_data["Q_O(s-1)"].values[0])
    / comp_data["Q_O(s-1)"].values[0],
)
print(
    "N injection rate:",
    N_injection_rate,
    comp_data["Q_N(s-1)"].values[0],
    (N_injection_rate - comp_data["Q_N(s-1)"].values[0])
    / comp_data["Q_N(s-1)"].values[0],
)

In [ ]:
from datetime import datetime
import numpy as np
from nrlmsise00 import msise_flat
alts = np.arange(20, 1000, 10.)  # = [200, 300, 400] [km]
# Using broadcasting, the output will be a 2 x 3 x 11 element array:
msise_flat(datetime(2009, 6, 21, 8, 3, 20), alts[None, :], 0, -70, 150, 150, 4)

plt.figure(figsize=(8, 6))
gs = plt.GridSpec(2, 2)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[0, 1])
ax2 = plt.subplot(gs[1, 0])
ax3 = plt.subplot(gs[1, 1])

for alt in alts:
    dens = msise_flat(datetime(2025, 12, 9, 8, 3, 20), np.array([[alt]]), 0, -70, 150, 150, 4) * 1e6  # convert from cm^-3 to m^-3
    # alt = alt * 1e3  # convert to m
    ax0.semilogy(alt, dens[0,0,3], marker="o", color="C0")  # O2
    ax1.semilogy(alt, dens[0,0,2], marker="o", color="C1")  # N2
    ax2.semilogy(alt, dens[0,0,1], marker="o", color="C2")  # O
    ax3.semilogy(alt, dens[0,0,7], marker="o", color="C3")  # N

ax0.semilogy(
    comp_data["Heit(km)"], comp_data["O2den(m-3)"], label="COMPAT"
)

ax1.semilogy(
    comp_data["Heit(km)"], comp_data["N2den(m-3)"], label="COMPAT"
)
ax2.semilogy(
    comp_data["Heit(km)"], comp_data["Oden(m-3)"], label="COMPAT"
)
ax3.semilogy(
    comp_data["Heit(km)"], comp_data["Nden(m-3)"], label="COMPAT"
)

for ax in [ax0, ax1, ax2, ax3]:
    ax.set_xlabel("Altitude [km]")
    ax.set_ylabel("Density [m$^{-3}$]")
    ax.legend()
    ax.grid(ls = ":")
    # ax.set_ylim(1e4, 1e25)


In [ ]:
print(comp_data.head())

In [ ]:
plt.figure()
plt.plot(comp_data['Heit(km)'], comp_data['T(eV)'])
interpolate_Te = np.interp(alts, comp_data['Heit(km)'], comp_data['T(eV)'])
plt.plot(alts, interpolate_Te, marker="o")
plt.xlabel("Altitude [km]")
plt.ylabel("Temperature [eV]")

In [ ]:
O2_density = []
N2_density = []
O_density = []
N_density = []

for alt in alts:
    dens = msise_flat(datetime(2009, 6, 21, 8, 3, 20), np.array([[alt]]), 0, -70, 150, 150, 4) * 1e6  # convert from cm^-3 to m^-3
    # alt = alt * 1e3  # convert to m
    O2_density.append(dens[0,0,3])  # O2
    N2_density.append(dens[0,0,2])  # N2
    O_density.append(dens[0,0,1])  # O
    N_density.append(dens[0,0,7])  # N

    # ax0.semilogy(alt, dens[0,0,3], marker="o", color="C0")  # O2
    # ax1.semilogy(alt, dens[0,0,2], marker="o", color="C1")  # N2
    # ax2.semilogy(alt, dens[0,0,1], marker="o", color="C2")  # O
    # ax3.semilogy(alt, dens[0,0,7], marker="o", color="C3")  # N
AREA = 1.0  # m^2
orbital_speeds = np.sqrt(mu / (R_earth + altitude))

O2_injection_rate = orbital_speeds * np.array(O2_density) * AREA  # particles/s
N2_injection_rate = orbital_speeds * np.array(N2_density) * AREA  # particles/s
O_injection_rate = orbital_speeds * np.array(O_density) * AREA  # particles/s
N_injection_rate = orbital_speeds * np.array(N_density) * AREA  # particles/s

In [ ]:
plt.figure(figsize=(8, 6))
gs = plt.GridSpec(2, 2)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[0, 1])
ax2 = plt.subplot(gs[1, 0])
ax3 = plt.subplot(gs[1, 1])

ax0.plot(alts, O2_injection_rate, marker="o", label="NRLMSISE-00")
ax1.plot(alts, N2_injection_rate, marker="o", label="NRLMSISE-00")
ax2.plot(alts, O_injection_rate, marker="o", label="NRLMSISE-00")
ax3.plot(alts, N_injection_rate, marker="o", label="NRLMSISE-00")

ax0.plot(
    comp_data["Heit(km)"], comp_data["Q_O2(s-1)"], label="COMPAT"
)
ax1.plot(
    comp_data["Heit(km)"], comp_data["Q_N2(s-1)"], label="COMPAT"
)
ax2.plot(
    comp_data["Heit(km)"], comp_data["Q_O(s-1)"], label="COMPAT"
)
ax3.plot(
    comp_data["Heit(km)"], comp_data["Q_N(s-1)"], label="COMPAT"
)
for ax in [ax0, ax1, ax2, ax3]:
    ax.set_xlabel("Altitude [km]")
    ax.set_ylabel("Injection Rate [particles/s]")
    ax.legend()
    ax.grid(ls = ":")
    ax.set_yscale("log")

In [ ]:
# create a new database
database_new = pd.DataFrame({
    "Heit(km)": alts,
    'Oden(m-3)': O_density,
    'O2den(m-3)': O2_density,
    'Nden(m-3)': N_density,
    'N2den(m-3)': N2_density,
    'T(eV)': interpolate_Te,
    "Q_O2(s-1)": O2_injection_rate,
    "Q_N2(s-1)": N2_injection_rate,
    "Q_O(s-1)": O_injection_rate,
    "Q_N(s-1)": N_injection_rate,
})

print(database_new.head())

# database_new.to_csv("comp_atm_nrlmsise00_ready.txt", sep="\t", index=False)

In [ ]:
print(comp_data[comp_data["Heit(km)"] == 350.0])
print(database_new[database_new["Heit(km)"] == 350.0])


In [ ]:
altitudes = [0, 50, 100, 150, 200, 250, 300, 350, 400]

newO2 = []
compO2 = []
newN2 = []
compN2 = []
newO = []
compO = []
newN = []
compN = []
for alt in altitudes:
    #O2 density
    newO2.append(database_new[database_new["Heit(km)"] == alt]['O2den(m-3)'].values[0] if len(database_new[database_new["Heit(km)"] == alt]) > 0 else 0)
    compO2.append(comp_data[comp_data["Heit(km)"] == alt]['O2den(m-3)'].values[0] if len(comp_data[comp_data["Heit(km)"] == alt]) > 0 else 0)
    #N2 density
    newN2.append(database_new[database_new["Heit(km)"] == alt]['N2den(m-3)'].values[0] if len(database_new[database_new["Heit(km)"] == alt]) > 0 else 0)
    compN2.append(comp_data[comp_data["Heit(km)"] == alt]['N2den(m-3)'].values[0] if len(comp_data[comp_data["Heit(km)"] == alt]) > 0 else 0)
    #O density
    newO.append(database_new[database_new["Heit(km)"] == alt]['Oden(m-3)'].values[0] if len(database_new[database_new["Heit(km)"] == alt]) > 0 else 0)
    compO.append(comp_data[comp_data["Heit(km)"] == alt]['Oden(m-3)'].values[0] if len(comp_data[comp_data["Heit(km)"] == alt]) > 0 else 0)
    #N density
    newN.append(database_new[database_new["Heit(km)"] == alt]['Nden(m-3)'].values[0] if len(database_new[database_new["Heit(km)"] == alt]) > 0 else 0)
    compN.append(comp_data[comp_data["Heit(km)"] == alt]['Nden(m-3)'].values[0] if len(comp_data[comp_data["Heit(km)"] == alt]) > 0 else 0)
    
plt.figure(figsize=(10, 8))
plt.subplot(2,2,1)
plt.plot(altitudes, newO2, marker="o", label="NRLMSISE-00")
plt.plot(altitudes, compO2, marker="o", label="COMPAT")
plt.xlabel("Altitude [km]")
plt.ylabel("O2 Density [m$^{-3}$]")
plt.yscale("log")
plt.grid(ls=":")
plt.legend()
plt.subplot(2,2,2)
plt.plot(altitudes, newN2, marker="o", label="NRLMSISE-00")
plt.plot(altitudes, compN2, marker="o", label="COMPAT")
plt.xlabel("Altitude [km]")
plt.ylabel("N2 Density [m$^{-3}$]")
plt.yscale("log")
plt.grid(ls=":")
plt.legend()
plt.subplot(2,2,3)
plt.plot(altitudes, newO, marker="o", label="NRLMSISE-00")
plt.plot(altitudes, compO, marker="o", label="COMPAT")
plt.xlabel("Altitude [km]")
plt.ylabel("O Density [m$^{-3}$]")
plt.yscale("log")
plt.grid(ls=":")
plt.legend()
plt.subplot(2,2,4)
plt.plot(altitudes, newN, marker="o", label="NRLMSISE-00")
plt.plot(altitudes, compN, marker="o", label="COMPAT")
plt.xlabel("Altitude [km]")
plt.ylabel("N Density [m$^{-3}$]")
plt.yscale("log")
plt.grid(ls=":")
plt.legend()
plt.tight_layout()